In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [5]:
PINECONE_API_KEY = "774bb6f7-f6e4-4af5-92f8-546b931d7aaa"
PINECONE_API_ENV = "gcp-starter"

In [7]:
#Extract data from the pdf
def load_pdf(data):
    DirectoryLoader(data, 
                    glob=".pdf",
                    Loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents